## Import Libraries

In [ ]:
import os
import random
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D,Flatten, GlobalMaxPool2D, Dense

print("✅ Libraries Imported Successfully")

✅ Libraries Imported Successfully


## Prepare Data

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# unzip data folder

! unzip "/content/drive/MyDrive/learn&build/Machine_learning_classes/Deep Learning/cnn/food_10.zip"

Streaming output truncated to the last 5000 lines.
  inflating: food_10/french_fries/1020588.jpg  
  inflating: food_10/french_fries/1026314.jpg  
  inflating: food_10/french_fries/1033213.jpg  
  inflating: food_10/french_fries/1035421.jpg  
  inflating: food_10/french_fries/1049122.jpg  
  inflating: food_10/french_fries/10500.jpg  
  inflating: food_10/french_fries/1060646.jpg  
  inflating: food_10/french_fries/1062594.jpg  
  inflating: food_10/french_fries/1064632.jpg  
  inflating: food_10/french_fries/106608.jpg  
  inflating: food_10/french_fries/1071481.jpg  
  inflating: food_10/french_fries/107312.jpg  
  inflating: food_10/french_fries/1074446.jpg  
  inflating: food_10/french_fries/1078563.jpg  
  inflating: food_10/french_fries/1080215.jpg  
  inflating: food_10/french_fries/1081387.jpg  
  inflating: food_10/french_fries/1083531.jpg  
  inflating: food_10/french_fries/1085071.jpg  
  inflating: food_10/french_fries/108743.jpg  
  inflating: food_10/french_fries/1089500.

In [ ]:
os.listdir("/content/food_10")

['ice_cream',
 'french_fries',
 'apple_pie',
 'dumplings',
 'donuts',
 'garlic_bread',
 'cup_cakes',
 'waffles',
 'chocolate_cake',
 'samosa']

In [ ]:
filename = os.listdir(os.path.join("/content/food_10", "donuts"))
len(filename)

1000

In [ ]:
# finding number of images in each food category
for folder in os.listdir("/content/food_10"):
  file_name = os.listdir(os.path.join("/content/food_10", folder))
  print(f"{folder} containg {len(file_name)} images")

samosa containg 1000 images
apple_pie containg 1000 images
french_fries containg 1000 images
ice_cream containg 1000 images
garlic_bread containg 1000 images
chocolate_cake containg 1000 images
donuts containg 1000 images
dumplings containg 1000 images
waffles containg 1000 images
cup_cakes containg 1000 images


In [ ]:
os.path.split("/content/food_10/chocolate_cake")[-1]

'chocolate_cake'

In [ ]:
# split data in train test

import os
import shutil
import random

paths = {
    "data_path" : os.path.join(os.getcwd(),"data"),
    "train_path" : os.path.join(os.getcwd(),"data", "train"),
    "test_path" : os.path.join(os.getcwd(),"data","test")
}

def image_data_prep(img_dir, test_ratio):

  '''
  Takes image directory address as input
  img_dir: Category image folder address
  test_ratio : Ratio of total files you want in test must be in float (0,1)
  '''

  image_names = os.listdir(img_dir)
  food_cat = os.path.split(img_dir)[-1]

  #shuffle image_names
  random.seed(42)
  random.shuffle(image_names)

  num_test = round(len(image_names) * test_ratio)

  print(f'number of {food_cat} images in test folder {num_test}' )

  test_img = image_names[:num_test]
  train_img = image_names[num_test:]

  if not(os.path.exists(paths["data_path"])):
    os.mkdir(paths["data_path"])

  if not(os.path.exists(paths["train_path"])):
    os.mkdir(paths["train_path"])

  if not(os.path.exists(paths["test_path"])):
    os.mkdir(paths["test_path"])

  food_cat_train_path = os.path.join(paths["train_path"], food_cat)
  food_cat_test_path = os.path.join(paths["test_path"], food_cat)

  os.mkdir(food_cat_train_path)
  os.mkdir(food_cat_test_path)

  for img in test_img:
    src = os.path.join(img_dir, img)
    dst = food_cat_test_path
    shutil.move(src, dst)

  for img in train_img:
    src = os.path.join(img_dir, img)
    dst = food_cat_train_path
    shutil.move(src, dst)

In [ ]:
for folder in os.listdir("/content/food_10"):
  img_dir = os.path.join("/content/food_10", folder)
  image_data_prep(img_dir, test_ratio = 0.25)

number of samosa images in test folder 250
number of apple_pie images in test folder 250
number of french_fries images in test folder 250
number of ice_cream images in test folder 250
number of garlic_bread images in test folder 250
number of chocolate_cake images in test folder 250
number of donuts images in test folder 250
number of dumplings images in test folder 250
number of waffles images in test folder 250
number of cup_cakes images in test folder 250


## Preprocess and load data

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255)
test_datagen = ImageDataGenerator(rescale = 1.0/255)

train_data = train_datagen.flow_from_directory(
    paths["train_path"],
    target_size = (400,400),
    batch_size = 32,
    class_mode = "categorical",
    seed = 42
)

test_data = test_datagen.flow_from_directory(
    paths["test_path"],
    target_size = (400,400),
    batch_size = 32,
    class_mode = "categorical",
    seed = 42
)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
train_data.class_indices.keys()

dict_keys(['apple_pie', 'chocolate_cake', 'cup_cakes', 'donuts', 'dumplings', 'french_fries', 'garlic_bread', 'ice_cream', 'samosa', 'waffles'])

In [ ]:
batch = train_data.next()

In [ ]:
batch[0].shape, batch[1].shape

((32, 400, 400, 3), (32, 10))

In [ ]:
batch[0][0]

In [ ]:
batch[1][0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
model = tf.keras.models.Sequential([
    InputLayer(input_shape = (400,400,3)),
    Conv2D(filters = 32,
           kernel_size = (3,3),
           strides = (1,1),
           padding = "valid",
           activation = "relu"
           ),

    Conv2D(filters = 32,
           kernel_size = (3,3),
           strides = (1,1),
           padding = "valid",
           activation = "relu"
           ),
    MaxPool2D(),

    Conv2D(filters = 16,
           kernel_size = (3,3),
           strides = (1,1),
           padding = "valid",
           activation = "relu"
           ),

    Conv2D(filters = 16,
           kernel_size = (3,3),
           strides = (1,1),
           padding = "valid",
           activation = "relu"
           ),
    MaxPool2D(),

    GlobalMaxPool2D(),
    Dense(50, activation = "relu"),
    Dense(10, activation = "softmax")

])

In [ ]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

result = model.fit(train_data, epochs = 5, validation_data = test_data)

tensorflow hub - https://tfhub.dev/

copying feature vector url of efficientnet_v2 - https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2

In [ ]:
import tensorflow_hub as hub

model_layer = hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/feature_vector/2")

model_2 = tf.keras.models.Sequential([
    InputLayer(input_shape = (400,400,3)),
    model_layer,
    Dense(10, activation = "softmax")
])

model_2.compile(loss = "categorical_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

result = model_2.fit(train_data, epochs = 5, validation_data = test_data)

In [ ]:
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              20331360  
                                                                 
 dense_6 (Dense)             (None, 10)                12810     
                                                                 
Total params: 20344170 (77.61 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 20331360 (77.56 MB)
_________________________________________________________________


In [ ]:
tf.keras.save()

In [ ]:
! pip install streamlit

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import streamlit as st


st.write("# Food Classifier")


image_file = st.file_uploader("Upload Images", type=["png","jpg","jpeg"])

class_name = ['apple_pie', 'chocolate_cake',
              'cup_cakes', 'donuts', 'dumplings', 'french_fries',
              'garlic_bread', 'ice_cream', 'samosa', 'waffles']

def predict(model, img):
    img = img.resize((224,224))
    # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    y_prob = model.predict(tf.expand_dims(img, axis = 0))
    y_pred_index = np.argmax(y_prob, axis = 1)[0]
    y_pred = class_name[y_pred_index]
    return (f"Predicted food: {y_pred}, with probability: {round(y_prob.max()* 100, 2)}%")

# #load model
# @st.cache_resource
# def cache_model(model_name):
#     model = tf.keras.models.load_model(model_name)
#     return (model)

# model = cache_model("food_10_model")


if st.button("Predict"):

    img = Image.open(image_file)
    st.image(img, width=250)

    st.write(predict(model, img))